In [1]:
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
import tensorflow as tf
sess = tf.InteractiveSession()

In [6]:
knn__ = 12
batch_size__ = 32

uniforms_ = []
peaks_ = []

for i in range(1):

    if i % 10 == 0:
        print(i)
    
    # initialize exponential mass
    m_np = np.random.exponential(80., [batch_size__,1])+80.
    #m_np = np.array([120.,120.,120.,120.,120.,120.])
    #m_np = np.array([100.,150.,140.,130.,125.,110.])
    m__ = tf.constant(m_np, dtype=tf.float32)

    # init binary class
    #y_true_np = np.random.randint(2, size=batch_size__)
    y_true_np = np.zeros(batch_size__)
    y_true__ = tf.constant(y_true_np, dtype=tf.int32)

    # init preds
    y_pred_np = np.random.rand(batch_size__,1)
    #y_pred_np = np.array([0.6,0.6,0.6,0.6,0.6,0.6])
    #y_pred_np = np.array([0.1,0.2,.9,.8,0.5,0.6])
    # uniform distribution
    y_pred__ = tf.constant(y_pred_np, dtype=tf.float32)
    uniforms_.append(cvm_loss_grad(y_pred__, m__, y_true__, knn__, debug=False)[0].eval())
    print(uniforms_[-1])
    # peaked: artificially set masses near 125 to have y_pred=1.
    y_pred_np[(m_np > 115.) & (m_np < 135.)] = 1.
    y_pred__ = tf.constant(y_pred_np, dtype=tf.float32)
    peaks_.append(cvm_loss_grad(y_pred__, m__, y_true__, knn__, debug=False)[0].eval())
    print(peaks_[-1])

0
0.0142884
0.0271016


In [3]:
def sample_uniform(y_pred, m_i, m, debug=True):
    '''
    Sample a uniform CDF and order them according to y_pred
    '''
    # Get kNN of m_i #
    dist = tf.abs(tf.subtract(m_i, m))
    # Get index i
    _, indices_knn = tf.nn.top_k(tf.negative(dist), k=1)
    i = indices_knn[0]

    # Make a uniform pdf
    pdf = tf.ones_like(y_pred)
    cdf = tf.cumsum(pdf)
    #if debug: print('cdf',cdf.eval())
    cdf = tf.divide(cdf, cdf[-1])
    
    # Sort all y_preds
    _, indices_pred_sorted = tf.nn.top_k(tf.negative(y_pred), k=tf.shape(y_pred)[0])
    #if debug: print('indices_pred_sorted',indices_pred_sorted.eval())
    _, indices_orig = tf.nn.top_k(tf.negative(indices_pred_sorted), k=tf.shape(y_pred)[0])
    
    # Sample from cdf in same order as scores in y_pred
    cdf_pred = tf.gather(cdf, indices_orig)
    if debug: print('cdf_pred',cdf_pred.eval(),'i_',i.eval())
    
    return cdf_pred, i

In [4]:
def cvm_loss_grad_i(y_pred, m_i, m, K, debug=False):
    '''
    Calculate CVM gradient and loss contribution for the ith sample.
    CVM is expressed in terms of the CDF of the kNN(i) prediction scores s, F_knn(i)_(s)
    and of a reference uniform CDF, F_(s).
    
    Args:
        y_pred: array of prediction scores (including the ith sample)
        m_i: scalar representing mass of the ith sample
        m: array of mass values (including the ith sample)
        K: scalar representing number of nearest neighbors to use (including the ith sample)
    Returns:
        loss_i: contribution of the ith sample to the CVM loss
        grad_i: CVM gradient wrt y_pred_i
    
    '''
    if debug: print('m_i:',m_i.eval())
    
    y_pred = tf.squeeze(y_pred)
    m = tf.squeeze(m)
    
    # Sample CDF like y_pred
    cdf_pred_all, i_all = sample_uniform(y_pred, m_i, m, debug)
    
    # Get kNN of m_i #
    dist = tf.abs(tf.subtract(m_i, m))
    if debug: print('dist',dist.eval())
    # Sort by distance (i.e. by nearest neighbor)
    _, indices_knn = tf.nn.top_k(tf.negative(dist), k=K)
    if debug: print('indices_knn',indices_knn.eval())
    # Get corresponding y_preds and m for these kNN
    y_pred_knn = tf.gather(y_pred, indices_knn)
    if debug: print('y_pred_knn',y_pred_knn.eval())
    m_knn = tf.gather(m, indices_knn)
    if debug: print('m_knn',m_knn.eval())
    # Sample CDF like y_pred_knn
    cdf_pred_knn, i_knn = sample_uniform(y_pred_knn, m_i, m_knn, debug)
    
    # Get loss
    cdf_pred_all_knn = tf.gather(cdf_pred_all, indices_knn)
    integrand = tf.pow(tf.subtract(cdf_pred_knn, cdf_pred_all_knn),2.)
    loss_i = tf.reduce_mean(integrand)
    
    # Get grad
    grad_i = tf.abs(tf.subtract(cdf_pred_all[i_all],cdf_pred_knn[i_knn]))
    
    return tf.stack([loss_i, grad_i])

# Debug with smaller K
cvm_i = cvm_loss_grad_i(y_pred__, m__[4], m__, 4, debug=True)
print('loss_i:',cvm_i[0].eval(),' grad_i:',cvm_i[1].eval())

m_i: [ 110.97079468]
cdf_pred [ 0.4375   0.53125  0.5625   0.125    0.75     0.96875  0.25     0.21875
  0.40625  0.65625  0.03125  0.375    0.15625  0.8125   0.34375  0.6875
  0.46875  0.90625  0.84375  0.625    0.59375  1.       0.28125  0.09375
  0.1875   0.875    0.9375   0.5      0.3125   0.71875  0.0625   0.78125] i_ 4
dist [   6.66320038   17.14775848    4.89836121   18.22296143    0.
   27.47932434    6.67206573  267.69012451   74.61419678   21.17259216
    8.08588409   48.24597168   20.2829895   297.61682129   15.87670898
   93.71295166   95.46559143    4.89002228   12.07178497   39.24446106
   83.25579834   14.32927704    3.7778244    57.77528381   24.42472839
   15.23423767   11.66690063   13.19974518   29.72079468  111.90922546
   14.58078766  221.42474365]
indices_knn [ 4 22 17  2]
y_pred_knn [ 0.73169005  0.25472414  0.97287232  0.51047093]
m_knn [ 110.97079468  107.19297028  115.86081696  106.07243347]
cdf_pred [ 0.75  0.25  1.    0.5 ] i_ 0
loss_i: 0.00341797  grad_i: 0

In [5]:
def cvm_loss_grad(y_pred_, m_, y_true, K, debug=False):
    '''
    Calculate Cramer-von Mises loss and gradient (with kNN modification) for this batch.
    See: https://arxiv.org/pdf/1410.4140.pdf
    '''   
    # Only pick out bkg events (i.e. label=0)
    is_bkg = tf.equal(tf.cast(y_true, dtype=tf.int32), 0)
    y_pred = tf.boolean_mask(y_pred_, is_bkg)
    if debug: print('y_pred:',y_pred.eval())
    m = tf.boolean_mask(m_, is_bkg)
    if debug: print('m:',m.eval())
    
    cvm = tf.map_fn(lambda m_i: cvm_loss_grad_i(y_pred, m_i, m, K, debug=False), m, dtype=tf.float32)
    loss = tf.reduce_mean(cvm[:,0]) # assume each loss_i is equally weighted
    grad = cvm[:,1]
    
    #TODO: return grad for label=1 elements in y_pred as well (i.e. None)
    #grads = tf.zeros_like(y_pred_)
    #grads[is_bkg] = cvm[:,1]
    
    return loss, grad
    
# Debug
cvm = cvm_loss_grad(y_pred__, m__, y_true__, knn__, debug=True)
print('net loss:',cvm[0].eval())
print('grad wrt y_pred[is_bkg]:',cvm[1].eval())

y_pred: [[ 0.42541465]
 [ 0.50400704]
 [ 0.51047093]
 [ 0.01579145]
 [ 0.73169005]
 [ 0.99330503]
 [ 0.16287753]
 [ 0.12663478]
 [ 0.37483418]
 [ 0.69321942]
 [ 0.00290103]
 [ 0.36922905]
 [ 0.05867933]
 [ 0.78933609]
 [ 0.3497692 ]
 [ 0.70252371]
 [ 0.49122909]
 [ 0.97287232]
 [ 0.8359679 ]
 [ 0.61023736]
 [ 0.56451899]
 [ 0.99738914]
 [ 0.25472414]
 [ 0.01437786]
 [ 0.08904507]
 [ 0.93898261]
 [ 0.97345942]
 [ 0.49148154]
 [ 0.34093598]
 [ 0.72285759]
 [ 0.01089676]
 [ 0.7599532 ]]
m: [[ 104.3075943 ]
 [  93.82303619]
 [ 106.07243347]
 [ 129.1937561 ]
 [ 110.97079468]
 [ 138.45011902]
 [ 104.29872894]
 [ 378.66091919]
 [ 185.58499146]
 [  89.79820251]
 [ 119.05667877]
 [ 159.21676636]
 [  90.68780518]
 [ 408.58761597]
 [ 126.84750366]
 [ 204.68374634]
 [ 206.43638611]
 [ 115.86081696]
 [ 123.04257965]
 [ 150.21525574]
 [ 194.22659302]
 [  96.64151764]
 [ 107.19297028]
 [ 168.74607849]
 [ 135.39552307]
 [  95.73655701]
 [ 122.63769531]
 [  97.7710495 ]
 [ 140.69158936]
 [ 222.88002014